# Ariane Atlas – Interactive Exploration Notebook

This notebook shows how to use the Python SDK to talk to an Atlas server.

You can:

1. Check server health
2. List available contexts
3. Inspect states and transitions
4. Compute shortest paths between states
5. (Optionally) ingest a bundle generated by Theseus


## 1. Configuration

Set the base URL of your Atlas server and (optionally) an API key if auth is enabled.


In [ ]:
from consumers.sdk.client import AtlasClient, AtlasClientConfig, AtlasClientError
from consumers.sdk.types import ContextInfo, StateView, TransitionView, PathView

BASE_URL = "http://localhost:8080"  # change if needed
API_KEY = None                       # e.g. "your-secret-key" if auth is enabled

cfg = AtlasClientConfig(
    base_url=BASE_URL,
    api_key=API_KEY,
)
client = AtlasClient(cfg)

client  # show basic repr


## 2. Health check

Confirm that the server is reachable and the in-memory store is responsive.


In [ ]:
health = client.health()
health

## 3. List contexts

Contexts group UI maps by app/version/platform. This returns all known contexts.


In [ ]:
contexts = client.list_contexts()
for ctx in contexts:
    print(f"context_id: {ctx.context_id}")
    print(f"  app_id:  {ctx.app_id}")
    if ctx.version:
        print(f"  version: {ctx.version}")
    if ctx.platform:
        print(f"  platform: {ctx.platform}")
    if ctx.locale:
        print(f"  locale:   {ctx.locale}")
    print()

if not contexts:
    print("No contexts found. Ingest a bundle first (see later cell).")


Pick a context to work with. If you have only one, this cell selects it automatically.


In [ ]:
if not contexts:
    raise RuntimeError("No contexts available – ingest data first.")

# Simple choice: first context in the list
ctx: ContextInfo = contexts[0]
CONTEXT_ID = ctx.context_id
CONTEXT_ID

## 4. Inspect states

Fetch all states for the chosen context and inspect a few.


In [ ]:
states = client.list_states(CONTEXT_ID)
len(states), states[:3]  # number of states and a preview of the first three


Show details for the first state, including some interactive elements.


In [ ]:
first_state: StateView = states[0]
print("state_id:", first_state.state_id)
print("is_entry:", first_state.is_entry)
print("is_terminal:", first_state.is_terminal)
print("elements:")
for el in first_state.interactive_elements[:10]:
    label = f" label='{el.label}'" if el.label else ""
    print(f"  - [{el.role}] {el.id}{label}")

if len(first_state.interactive_elements) > 10:
    print(f"  ... +{len(first_state.interactive_elements) - 10} more element(s)")


## 5. Inspect transitions

Transitions represent edges between states, caused by user actions.


In [ ]:
transitions = client.list_transitions(CONTEXT_ID)
len(transitions), transitions[:3]  # count and preview of first three


## 6. Compute a shortest path between two states

Pick two state IDs. Below we simply choose the first and last state as an example.
You can override `SOURCE_STATE_ID` and `TARGET_STATE_ID` manually if you prefer.


In [ ]:
if len(states) < 2:
    raise RuntimeError("Need at least two states to compute a path.")

SOURCE_STATE_ID = states[0].state_id
TARGET_STATE_ID = states[-1].state_id

SOURCE_STATE_ID, TARGET_STATE_ID

In [ ]:
path = client.shortest_path(
    context_id=CONTEXT_ID,
    source_state_id=SOURCE_STATE_ID,
    target_state_id=TARGET_STATE_ID,
    max_depth=20,  # adjust as needed
)
path

Render the path step-by-step, with a human-readable summary of the actions.


In [ ]:
if path.transitions is None:
    print("No path found between the chosen states.")
elif not path.transitions:
    print("Source and target are the same state (empty path).")
else:
    print(f"Path length: {len(path.transitions)} transition(s)\n")
    for i, t in enumerate(path.transitions, start=1):
        print(f"Step {i}:")
        print(f"  from: {t.source_state_id}")
        print(f"  to:   {t.target_state_id}")
        print(f"  action.type: {t.action.type}")
        if t.action.element_id:
            print(f"  element_id:  {t.action.element_id}")
        if t.intent_id:
            print(f"  intent_id:   {t.intent_id}")
        print()


## 7. (Optional) Ingest a bundle from disk

If you have a bundle JSON file produced by Theseus (e.g. via `run_scan_to_disk`),
you can ingest it directly into Atlas.


In [ ]:
import json
from pathlib import Path

# Change this path to point to your bundle file, e.g.
# BUNDLE_PATH = Path("./out/2025-01-01T12-00-00Z/atlas_bundle.json")
BUNDLE_PATH = Path("./atlas_bundle.json")

if BUNDLE_PATH.exists():
    with BUNDLE_PATH.open("r", encoding="utf-8") as f:
        bundle = json.load(f)

    response = client.ingest_bundle(bundle)
    response
else:
    print(f"Bundle path does not exist: {BUNDLE_PATH}")
